<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [30]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [31]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [32]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#0000FF', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [33]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, site in launch_sites_df.iterrows():
    circle = folium.Circle([site['Lat'], site['Long']], radius=1000, color='#0000FF', fill=True).add_child(folium.Popup(site['Launch Site']))
    marker = folium.map.Marker(
        [site['Lat'], site['Long']],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [34]:
# Task 2: Mark the success/failed launches for each site on the map
for index, record in spacex_df.iterrows():
    if record['class'] == 1:
        icon_color = 'green'
    else:
        icon_color = 'red'
    marker = folium.map.Marker([record['Lat'], record['Long']], icon=folium.Icon(color=icon_color))
    site_map.add_child(marker) 
    site_map.add_child(marker) 
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [35]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [36]:
marker_cluster = MarkerCluster()


In [37]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long','class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [38]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [39]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for i in range(spacex_df.shape[0]):
    folium.Marker([spacex_df.iloc[i]['Lat'], spacex_df.iloc[i]['Long']], icon=folium.Icon(color='white', icon_color=spacex_df.iloc[i]['marker_color'])).add_to(marker_cluster)

site_map.add_child(marker_cluster)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [41]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, sin, cos, sqrt, asin

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) * sin(dlat / 2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) * sin(dlon / 2)
    c = 2 * asin(sqrt(a))
    distance = R * c
    return distance

# Coordinates of the Kennedy Space Center
ksc_coordinate = (28.573255, -80.646895)
# Coordinates of the Vandenberg Air Force Base
vafb_coordinate = (34.632834, -120.610746)
# Coordinates of the Cape Canaveral Space Launch Complex
ccslc_coordinate = (28.562302, -80.577356)
# Coordinates of the SpaceX South Texas Launch Site
stls_coordinate = (25.997264, -97.156084)

# Distance from Kennedy Space Center to its proximities
ksc_vafb_dist = haversine(ksc_coordinate[0], ksc_coordinate[1], vafb_coordinate[0], vafb_coordinate[1])
ksc_ccslc_dist = haversine(ksc_coordinate[0], ksc_coordinate[1], ccslc_coordinate[0], ccslc_coordinate[1])
ksc_stls_dist = haversine(ksc_coordinate[0], ksc_coordinate[1], stls_coordinate[0], stls_coordinate[1])
ksc_vafb_dist, ksc_ccslc_dist, ksc_stls_dist
# Distance from Vandenberg Air Force Base to its proximities
vafb_ksc_dist = haversine(vafb_coordinate[0], vafb_coordinate[1], ksc_coordinate[0], ksc_coordinate[1])
vafb_ccslc_dist = haversine(vafb_coordinate[0], vafb_coordinate[1], ccslc_coordinate[0], ccslc_coordinate[1])
vafb_stls_dist = haversine(vafb_coordinate[0], vafb_coordinate[1], stls_coordinate[0], stls_coordinate[1])
vafb_ksc_dist, vafb_ccslc_dist, vafb_stls_dist

# Distance from Cape Canaveral Space Launch Complex to its proximities
ccslc_ksc_dist = haversine(ccslc_coordinate[0], ccslc_coordinate[1], ksc_coordinate[0], ksc_coordinate[1])
ccslc_vafb_dist = haversine(ccslc_coordinate[0], ccslc_coordinate[1], vafb_coordinate[0], vafb_coordinate[1])
ccslc_stls_dist = haversine(ccslc_coordinate[0], ccslc_coordinate[1], stls_coordinate[0], stls_coordinate[1])
ccslc_ksc_dist, ccslc_vafb_dist, ccslc_stls_dist

# Distance from SpaceX South Texas Launch Site to its proximities
stls_ksc_dist = haversine(stls_coordinate[0], stls_coordinate[1], ksc_coordinate[0], ksc_coordinate[1])
stls_vafb_dist = haversine(stls_coordinate[0], stls_coordinate[1], vafb_coordinate[0], vafb_coordinate[1])
stls_ccslc_dist = haversine(stls_coordinate[0], stls_coordinate[1], ccslc_coordinate[0], ccslc_coordinate[1])
stls_ksc_dist, stls_vafb_dist, stls_ccslc_dist

# Create a DataFrame to store the distances between the launch sites and their proximities
data = {'Launch Site': ['KSC', 'VAFB', 'CCSLC', 'STLS'],
        'KSC': [0, ksc_vafb_dist, ksc_ccslc_dist, ksc_stls_dist],
        'VAFB': [vafb_ksc_dist, 0, vafb_ccslc_dist, vafb_stls_dist],
        'CCSLC': [ccslc_ksc_dist, ccslc_vafb_dist, 0, ccslc_stls_dist],
        'STLS': [stls_ksc_dist, stls_vafb_dist, stls_ccslc_dist, 0]}
proximity_df = pd.DataFrame(data)
proximity_df.set_index('Launch Site', inplace=True)
proximity_df




,KSC,VAFB,CCSLC,STLS
Launch Site,,,,
KSC,0.000000,3819.052850,6.899330,1655.019251
VAFB,3819.052850,0.000000,3825.840368,2439.919315
CCSLC,6.899330,3825.840368,0.000000,1661.644570
STLS,1655.019251,2439.919315,1661.644570,0.000000


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [42]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [43]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [44]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
coastline = {
        'Florida': (28.5, -81.3),
        'VAFB': (34.6, -120.6),
        'Texas': (26.1, -97.3)
    }

def find_closest_coastline(launch_site):   
    
    # calculate distance between launch site to each coastline
    distances = {}
    for key, value in coastline.items():
        distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
        
    # find the closest coastline
    closest_coastline = min(distances, key=distances.get)
    return closest_coastline

# find the closest coastline to Kennedy Space Center
ksc_coastline = find_closest_coastline(ksc_coordinate)
ksc_coastline

'Florida'

In [46]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example

coastline_coordinate = coastline[ksc_coastline]
marker = folium.map.Marker(
    [coastline_coordinate[0], coastline_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_coastline,
        )
    )
site_map.add_child(marker)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [47]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
lines=folium.PolyLine(locations=[ksc_coordinate, coastline_coordinate], weight=1)
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [48]:
# find the closest city to Kennedy Space Center
city = {
        'Orlando': (28.5383, -81.3792),
        'Los Angeles': (34.0522, -118.2437),
        'Brownsville': (25.9018, -97.4975)
    }

def find_closest_city(launch_site):   
    
    # calculate distance between launch site to each city
    distances = {}
    for key, value in city.items():
        distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
        
    # find the closest city
    closest_city = min(distances, key=distances.get)
    return closest_city

# find the closest city to Kennedy Space Center
ksc_city = find_closest_city(ksc_coordinate)
ksc_city

# Create and add a folium.Marker on your selected closest city point on the map
city_coordinate = city[ksc_city]
marker = folium.map.Marker(
    [city_coordinate[0], city_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_city,
        )
    )
site_map.add_child(marker)
site_map

# Draw a line between the marker to the launch site
lines=folium.PolyLine(locations=[ksc_coordinate, city_coordinate], weight=1)
site_map.add_child(lines)
site_map

In [49]:
# find the closest railway to Kennedy Space Center      
railway = {
        'Cape Canaveral Railway': (28.3922, -80.6077),
        'Vandenberg Railway': (34.6328, -120.6107),
        'Brownsville Railway': (25.9018, -97.4975)
    }

def find_closest_railway(launch_site):   
    
    # calculate distance between launch site to each railway
    distances = {}
    for key, value in railway.items():
        distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
        
    # find the closest railway
    closest_railway = min(distances, key=distances.get)
    return closest_railway

# find the closest railway to Kennedy Space Center
ksc_railway = find_closest_railway(ksc_coordinate)
ksc_railway

# Create and add a folium.Marker on your selected closest railway point on the map
railway_coordinate = railway[ksc_railway]
marker = folium.map.Marker(
    [railway_coordinate[0], railway_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_railway,
        )
    )
site_map.add_child(marker)
site_map

# Draw a line between the marker to the launch site 
lines=folium.PolyLine(locations=[ksc_coordinate, railway_coordinate], weight=1)
site_map.add_child(lines)
site_map

In [50]:
# find the closest highway to Kennedy Space Center

highway = {
        'Florida State Road 528': (28.4107, -80.7370),
        'Interstate H-1': (34.6328, -120.6107),
        'Texas State Highway 4': (25.9018, -97.4975)
    }

def find_closest_highway(launch_site):
        
        # calculate distance between launch site to each highway
        distances = {}
        for key, value in highway.items():
            distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
            
        # find the closest highway
        closest_highway = min(distances, key=distances.get)
        return closest_highway

# find the closest highway to Kennedy Space Center
ksc_highway = find_closest_highway(ksc_coordinate)
ksc_highway

# Create and add a folium.Marker on your selected closest highway point on the map
highway_coordinate = highway[ksc_highway]
marker = folium.map.Marker(
    [highway_coordinate[0], highway_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_highway,
        )
    )
site_map.add_child(marker)
site_map

# Draw a line between the marker to the launch site     
lines=folium.PolyLine(locations=[ksc_coordinate, highway_coordinate], weight=1)
site_map.add_child(lines)
site_map

In [51]:
# find the closest coastline to Kennedy Space Center    
coastline = {
        'Florida': (28.5, -81.3),
        'VAFB': (34.6, -120.6),
        'Texas': (26.1, -97.3)
    }

def find_closest_coastline(launch_site):   
    
    # calculate distance between launch site to each coastline
    distances = {}
    for key, value in coastline.items():
        distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
        
    # find the closest coastline
    closest_coastline = min(distances, key=distances.get)
    return closest_coastline

# find the closest coastline to Kennedy Space Center
ksc_coastline = find_closest_coastline(ksc_coordinate)
ksc_coastline

# Create and add a folium.Marker on your selected closest coastline point on the map
coastline_coordinate = coastline[ksc_coastline]
marker = folium.map.Marker(
    [coastline_coordinate[0], coastline_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_coastline,
        )
    )
site_map.add_child(marker)
site_map

# Draw a line between the marker to the launch site  
lines=folium.PolyLine(locations=[ksc_coordinate, coastline_coordinate], weight=1)
site_map.add_child(lines)
site_map

In [52]:
# find the closest airport to Kennedy Space Center  
airport = {
        'Orlando International Airport': (28.4294, -81.3089),
        'Los Angeles International Airport': (33.9416, -118.4085),
        'Brownsville/South Padre Island International Airport': (25.9060, -97.4253)
    }

def find_closest_airport(launch_site):
            
            # calculate distance between launch site to each airport
            distances = {}
            for key, value in airport.items():
                distances[key] = calculate_distance(launch_site[0], launch_site[1], value[0], value[1])
                
            # find the closest airport
            closest_airport = min(distances, key=distances.get)
            return closest_airport

# find the closest airport to Kennedy Space Center
ksc_airport = find_closest_airport(ksc_coordinate)
ksc_airport

# Create and add a folium.Marker on your selected closest airport point on the map
airport_coordinate = airport[ksc_airport]
marker = folium.map.Marker(
    [airport_coordinate[0], airport_coordinate[1]],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % ksc_airport,
        )
    )
site_map.add_child(marker)
site_map

# Draw a line between the marker to the launch site
lines=folium.PolyLine(locations=[ksc_coordinate, airport_coordinate], weight=1)
site_map.add_child(lines)
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
